Дипломный проект

Модель прогнозирования стоимости жилья для агентства недвижимости

**3 Этап. Создание модели**

загружаем необходимые библиотеки

In [63]:
import random
import numpy as np 
import pandas as pd 
import sys
import optuna

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold
from sklearn import metrics
from tqdm.notebook import tqdm
from category_encoders import TargetEncoder, CatBoostEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

План работы:

-Обработаем и пронормируем признаки
-Построим "наивную" модель, предсказывающую цену по общей площади и городу (с ней будем сравнивать другие модели)
-Построим модель при помощи LinearRegression
-Обучим модель на основе случайного леса RandomForestRegressor
-Обучим модель с L1 и L2 регуляризаций ElasticNetCV
-На основе предыдущих шагов выберем оптимальную модель

In [64]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42
TEST_SIZE = 0.2

In [65]:
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
#отключим оповещения
import warnings
warnings.filterwarnings("ignore")

In [66]:
df = pd.read_csv('data_model.csv')
display(df.head())
df.info()

,status,baths,city,sqft,state,target,pool,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,4.0,Southern Pines,2900,NC,418000,0,single_family_home,2019,1,0,0,1,5.2,2.70
1,For Sale,3.0,Spokane Valley,1947,WA,310000,0,single_family_home,2019,0,0,0,0,4.0,1.01
2,Active,2.0,Mason,3588,IA,244900,0,single_family_home,1970,1,1,0,0,3.8,5.60
3,Other,3.0,Houston,1930,TX,311995,0,single_family_home,2019,1,1,1,0,3.0,0.60
4,For Sale,2.0,Flushing,1300,NY,669000,0,condo,1965,0,0,1,0,6.7,0.30


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210668 entries, 0 to 210667
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               210668 non-null  object 
 1   baths                210668 non-null  float64
 2   city                 210668 non-null  object 
 3   sqft                 210668 non-null  int64  
 4   state                210668 non-null  object 
 5   target               210668 non-null  int64  
 6   pool                 210668 non-null  int64  
 7   Type                 210668 non-null  object 
 8   Year built           210668 non-null  object 
 9   Heating_encoded      210668 non-null  int64  
 10  Cooling_encoded      210668 non-null  int64  
 11  Parking_encoded      210668 non-null  int64  
 12  fireplace_encoded    210668 non-null  int64  
 13  school_rating _mean  210668 non-null  float64
 14  school_dist_min      210668 non-null  float64
dtypes: float64(3), in

In [67]:
# Составим список булевых признаков:
bin_features = ['pool','Heating_encoded','Cooling_encoded','Parking_encoded','fireplace_encoded']

# Составим список категориальных признаков:
cat_features = ['status','city','state','Type','Year built']
 
# Составим список числовых признаков:
num_features = ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']

In [68]:
# подсчет количества уникальных значений в каждой категориальной колонке
for col in cat_features:
   unique_values = df[col].nunique()
   print(f"Количество уникальных значений в категориальной колонке {col}: {unique_values}")

Количество уникальных значений в категориальной колонке status: 12
Количество уникальных значений в категориальной колонке city: 1522
Количество уникальных значений в категориальной колонке state: 34
Количество уникальных значений в категориальной колонке Type: 12
Количество уникальных значений в категориальной колонке Year built: 205


In [69]:
def preproc_data(df_input):
    
    df_output = df_input.copy()
    # переведем признак год в категориальный
    df_output['Year built'] = df_output['Year built'].astype(str)
    # Нормализация и логорифмирование данных
    #scaler = MinMaxScaler()
    for column in ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']:
        # Логорифмирование
        df_output[column] = df_output[column].apply(lambda x: abs(x))
        constant = 1e-6
        df_output[column] = np.log(df_output[column] + constant)
    # категориальные признаки
 
    ohe_status = OneHotEncoder(sparse=False)
    ohe_state = OneHotEncoder(sparse=False)
    ohe_Type = OneHotEncoder(sparse=False)

    status_ohe = ohe_status.fit_transform(df_output['status'].values.reshape(-1,1))
    state_ohe = ohe_state.fit_transform(df_output['state'].values.reshape(-1,1))
    Type_ohe = ohe_Type.fit_transform(df_output['Type'].values.reshape(-1,1))

    le = LabelEncoder()
    state_label = le.fit_transform(df_output['state'])

    year_le = LabelEncoder()
    year_ord = year_le.fit_transform(df_output['Year built'])

    city_le = LabelEncoder()
    city_label = city_le.fit_transform(df_output['city'])
 
# добавление закодированных категориальных объектов в базу данных
    df_output = df_output.join(pd.DataFrame(status_ohe, columns=['status_' + str(cat) for cat in ohe_status.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(state_ohe, columns=['state_' + str(cat) for cat in ohe_state.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(Type_ohe, columns=['Type_' + str(cat) for cat in ohe_Type.categories_[0]]))
    df_output['state_label'] = state_label
    df_output['year_ord'] = year_ord
    df_output['city_label'] = city_label
   
# удаление исходных категориальных признаков
    df_output.drop(['status', 'state', 'Type', 'city', 'Year built'], axis=1, inplace=True)
    
    return df_output    
    

In [70]:
# Запускаем и проверяем, что получилось
df_encoded = preproc_data(df)
df_encoded.sample(10)

,baths,sqft,target,pool,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min,...,Type_mobile_home,Type_modern,Type_multi_family_home,Type_other,Type_ranch,Type_single_family_home,Type_townhouse,state_label,year_ord,city_label
14106,0.693148,7.060476,12.254387,0,1,1,1,1,1.791760,0.924259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,180,56
130234,0.693148,6.907755,13.422468,0,0,0,0,0,1.871802,-1.514123,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,186,163
146389,1.386295,7.583248,12.834655,0,1,1,1,0,1.193923,-0.105359,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,28,204,608
94660,1.098613,7.261927,12.273264,0,1,1,1,0,1.740466,-0.105359,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28,201,1182
189611,1.098613,7.914983,13.060275,1,1,1,1,1,1.987874,0.350658,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,27,161,890
137559,0.693148,7.254178,13.526494,1,1,1,1,0,2.014903,0.741938,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5,138,298
122204,0.693148,7.286192,11.951180,0,1,1,1,1,0.405466,-0.105359,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,14,111,339
208968,0.693148,7.532088,12.505807,0,1,1,1,0,1.308333,-0.544725,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5,201,1001
200295,1.386295,8.034955,12.846610,0,1,1,1,1,2.079442,-1.609433,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28,188,1182
93939,0.693148,7.740664,12.300928,0,1,1,1,0,1.987874,0.647104,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,27,138,258


In [71]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210668 entries, 0 to 210667
Data columns (total 71 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   baths                    210668 non-null  float64
 1   sqft                     210668 non-null  float64
 2   target                   210668 non-null  float64
 3   pool                     210668 non-null  int64  
 4   Heating_encoded          210668 non-null  int64  
 5   Cooling_encoded          210668 non-null  int64  
 6   Parking_encoded          210668 non-null  int64  
 7   fireplace_encoded        210668 non-null  int64  
 8   school_rating _mean      210668 non-null  float64
 9   school_dist_min          210668 non-null  float64
 10  status_Active            210668 non-null  float64
 11  status_Auction           210668 non-null  float64
 12  status_Back on Market    210668 non-null  float64
 13  status_Coming Soon       210668 non-null  float64
 14  stat

In [72]:
# Разделим датасет на обучающую и тестовую часть

y = df_encoded.target.values
X = df_encoded.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

Обучение модели

Model 1: Создадим "наивную" модель

Эта модель будет предсказывать среднюю стоимость по общей площади и городу. C ней будем сравнивать другие модели.

In [73]:
# Наивная модель
class NaiveModel:
    def __init__(self):
        self.means = None

    def fit(self, X, y):
        X_df = pd.DataFrame(X, columns=['city_label'])
        y_df = pd.DataFrame(y, columns=['target'])
        df = pd.concat([X_df, y_df], axis=1)
        self.means = df.groupby(['city_label'])['target'].mean().reset_index()

    def predict(self, X):
        X = pd.DataFrame(X, columns=['city_label']).copy()
        X['mean'] = np.nan
        for idx, row in self.means.iterrows():
            X.loc[(X['city_label'] == row['city_label']), 'mean'] = row['target']
        
        X['mean'].fillna(X['mean'].mean(), inplace=True)
        return X['mean'].to_numpy()

naive_model = NaiveModel()
naive_model.fit(X_train, y_train)
y_pred_train = naive_model.predict(X_train)
y_pred_test = naive_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.43
Test MSE: 0.42
Train MAE: 0.48
Test MAE: 0.47
Train R2: -0.01
Test R2: -0.01


MAE (Средняя абсолютная ошибка, Mean Absolute Error) - это мера ошибки, вычисленная как среднее значение абсолютных значений ошибок. Меньшие значения MAE указывают на лучшую точность модели. В данном случае:

Train MAE: 0.48 - это средняя абсолютная ошибка на обучающей выборке. Test MAE: 0.47 - это средняя абсолютная ошибка на тестовой выборке. R2 (коэффициент детерминации) - статистическая мера, которая показывает, насколько хорошо вариации зависимой переменной объясняются моделью. Значения R2 находятся в диапазоне от -∞ до 1. Чем ближе значение R22 к 1, тем лучше модель объясняет зависимость между переменными. В данном случае:

Train R2: -0.01 - это коэффициент детерминации на обучающей выборке. Test R2: -0.01 - это коэффициент детерминации на тестовой выборке. Проанализировав результаты, можно сделать следующие выводы:

Значения MSE и MAE, как для обучающей, так и для тестовой выборок, практически одинаковы. Это говорит о том, что модель демонстрирует схожую производительность на обеих выборках и не страдает от явного переобучения или недообучения.

Значения R^2 близки к нулю, что говорит о том, что модель слабо объясняет зависимость между переменными. Это может свидетельствовать о низкой предсказательной способности модели.

Model 2: LinearRegression

In [74]:
# создаём модель линейной регрессии
model = LinearRegression(fit_intercept=False)

# вычисляем коэффициенты регрессии
model.fit(X_train, y_train)

# делаем предсказания с помощью модели
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# вычисляем требуемые метрики
mse_train = metrics.mean_squared_error(y_train, y_train_pred)
mse_test = metrics.mean_squared_error(y_test, y_test_pred)
mae_train = metrics.mean_absolute_error(y_train, y_train_pred)
mae_test = metrics.mean_absolute_error(y_test, y_test_pred)
r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)

# выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.23
Test MSE: 0.22
Train MAE: 0.34
Test MAE: 0.34
Train R2: 0.47
Test R2: 0.46


Mean Squared Error, MSE - в данном случае,  для обучающей выборки составляет 0.23,  для тестовой выборки MSE составляет 0.22. Низкие значения MSE указывают на более точные предсказания модели.

Mean Absolute Error, MAE - в данном случае MAE составляет 0.34 для обучающей и 0.34 для тестовой выборки. Низкие значения MAE также говорят о хорошей точности предсказания.

Коэффициент детерминации (R2) В данном случае R2 составляет 0.47 как для обучающей, так и для тестовой выборки, что означает, что модель объясняет 47% дисперсии зависимой переменной.

Результаты модели показывают, что она работает с одинаковой точностью на обучающем и тестовом наборе данных, что свидетельствует о том, что модель не переобучилась. Однако, коэффициент детерминации R2 указывает на то, что есть еще большой потенциал для улучшения модели, поскольку ходят возможны другие альтернативы, такие, как увеличение размера выборки, использование других переменных или алгоритмов для еще большей предсказательной точности.

Model 3: RandomForestRegressor

In [75]:
# Создаем экземпляр модели RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=RANDOM_SEED)

# Обучаем модель на обучающих данных
rf_regressor.fit(X_train, y_train)

# Предсказания на обучающих и тестовых данных
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

# Вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.01
Test MSE: 0.08
Train MAE: 0.07
Test MAE: 0.19
Train R2: 0.97
Test R2: 0.79


Среднеквадратическая ошибка (MSE) - В данном случае, значение MSE для обучающей выборки составляет 0.01, тогда как для тестовой выборки - 0.08. Такое различие может указывать на небольшое переобучение модели.

Средняя абсолютная ошибка (MAE) - В данном случае, значения MAE составили 0.07 для обучающей выборки и 0.19 для тестовой выборки. Разница между этими значениями также может указывать на переобучение.

Коэффициент детерминации (R2) - В данном случае, R2 равен 0.97 для обучающей выборки и 0.79 для тестовой выборки. Эти значения указывают на то, что модель довольно хорошо работает на обучающей выборке и неплохо предсказывает на тестовой выборке, хотя значение R2 на тестовой выборке заметно ниже, что также может указывать на переобучение модели.

В целом, результаты показывают, что модель имеет хорошую производительность на обучающей выборке, но на тестовой выборке результаты хуже, что может свидетельствовать о переобучении.

Model 4: ElasticNetCV

In [76]:
# Создаем и тренируем модель ElasticNetCV с кросс-валидацией по 5 фолдам
model_el = ElasticNetCV(cv=5, random_state=RANDOM_SEED)
model_el.fit(X_train, y_train)

# Предсказания для обучающей и тестовой выборок
y_train_pred = model_el.predict(X_train)
y_test_pred = model_el.predict(X_test)

# MSE
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# MAE
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# R2
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.29
Test MSE: 0.28
Train MAE: 0.38
Test MAE: 0.38
Train R2: 0.31
Test R2: 0.31


MSE (Mean Squared Error) - В данном случае обучающая выборка имеет значение 0.29, а тестовая - 0.28. Эти числа близки, что означает, что модель не переобучилась, но в то же время обобщающая способность модели может быть не очень хорошей из-за высоких значений ошибок.

MAE (Mean Absolute Error) - В данном случае средняя абсолютная ошибка составляет 0.38 как для обучающей, так и для тестовой выборки. Это означает, что в среднем модель ошибается на 0.38 при предсказаниях.

R2 (коэффициент детерминации) - В данном случае R2 равен 0.31 для обучающей выборки и 0.31 для тестовой выборки. Эти значения средние, что говорит о низком качестве модели и низкой способности объяснить изменчивость данных.

Вывод: по представленным результатам видно, что модель имеет среднюю ошибку и средний коэффициент детерминации. Это означает, что модель не является оптимальной.

In [77]:
data = {'Metric': ['Train MSE', 'Test MSE', 'Train MAE', 'Test MAE', 'Train R2', 'Test R2'],
        'NaiveModel': [0.43, 0.42, 0.48, 0.47, -0.01, -0.01],
        'LinearRegression': [0.23, 0.22, 0.34, 0.34, 0.47, 0.46],
        'RandomForestRegressor': [0.01, 0.08, 0.07, 0.19, 0.97, 0.79],
        'ElasticNetCV': [0.29, 0.28, 0.38, 0.38, 0.31, 0.31]}

df_metric = pd.DataFrame(data)
df_metric.head(6)

,Metric,NaiveModel,LinearRegression,RandomForestRegressor,ElasticNetCV
0,Train MSE,0.43,0.23,0.01,0.29
1,Test MSE,0.42,0.22,0.08,0.28
2,Train MAE,0.48,0.34,0.07,0.38
3,Test MAE,0.47,0.34,0.19,0.38
4,Train R2,-0.01,0.47,0.97,0.31
5,Test R2,-0.01,0.46,0.79,0.31


На основе этих результатов, RandomForestRegressor кажется наилучшей моделью, так как она имеет наименьший показатель ошибок MSE и MAE, вместе с высоким коэффициентом детерминации R2 (0.79) на тестовых данных.

In [78]:
import pickle

# Сохранение выбранной обученной модели в файл pickle
with open('C:/Users/Илья/Documents/GitHub/Vera_data_science/Skillfactory/Финальный проект/data/best_rf_regressor_model.pkl', "wb") as f:
    pickle.dump(rf_regressor, f)

**Далее:**
*Подгрузим обученную модель rf_regressor*
*Напишем функцию для предсказания стоимости недвижимости по входным параметрам*

In [79]:
import numpy as np 
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings("ignore")


In [80]:
# Загрузка сохраненной модели из файла pickle
with open("data/best_rf_regressor_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

In [82]:
# создадим тестовый набор 
data = [
    ('Active',3.0, 'Washington', 801,  'DC', 0, 'other', 1991, 0, 0, 0, 0, 6.0, 0.2),
    ( 'Active',2.0, 'Dallas', 832,  'TX', 0, 'condo', 1998, 0, 1, 0, 0, 2.6, 0.6)    
]

columns = ['status', 'baths', 'city', 'sqft',  'state', 'pool', 'Type', 'Year built', 'Heating_encoded', 'Cooling_encoded', 'Parking_encoded', 'fireplace_encoded', 'school_rating _mean', 'school_dist_min']

df_test = pd.DataFrame(data, columns=columns)

df_test.head()

,status,baths,city,sqft,state,pool,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,3.0,Washington,801,DC,0,other,1991,0,0,0,0,6.0,0.2
1,Active,2.0,Dallas,832,TX,0,condo,1998,0,1,0,0,2.6,0.6


In [93]:

# функция принимает датасет и модель, затем обрабатывет датасет и передает его в модель для получения предсказания, предсказание выводится в нормальном виде
def preprocess_and_predict(df_input, model):
    
    def log_data(df_input):
        df_output = df_input.copy()
        df_output['Year built'] = df_output['Year built'].astype(str)
        
        #scaler = MinMaxScaler()
        for column in ['baths', 'sqft', 'school_rating _mean', 'school_dist_min']:
            #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
            df_output[column] = df_output[column].apply(lambda x: abs(x))
            constant = 1e-6
            df_output[column] = np.log(df_output[column] + constant)
        return df_output

    X_test = log_data(df_input)
    y_test_pred_loaded = model.predict(X_test)
    target = np.exp(y_test_pred_loaded)
    rounded_target = np.round(target)
    print(rounded_target)
    
    return rounded_target

In [ ]:
predictions = preprocess_and_predict(df_test, loaded_model)


[506389. 224225.]

Зафиксируем зависимости в файле requirements.txt.

In [90]:

!pip freeze > requirements.txt

**Заключение**

В итоге получена рабочая модель по предсказанию цены недвижимости по задаваемым пользователем параметрам.

Файлы сервера и клиента для реализации веб-сервиса по обработке запросов прилагаются.


Работу над проектом можно считать оконченой.

P.S/провести контейнеризацию с помощью Docker не представляется возможным из-за наличия проблем работы с ним на ОС Windows